In [36]:
import pandas as pd
import math
import zipfile
%xmode minimal

Exception reporting mode: Minimal


# Settings

In [37]:
# file containing master data and respective settings
masterdata = 'masterdata.xlsx'

In [38]:
# cover the following years
start_date = '2020-01-01'
end_date = '2034-01-01'

start_date = '2022-10-11' # test
end_date = '2022-12-24'   # test
start_date = '2022-12-11' # test
end_date = '2023-01-09'   # test

### Seasonality

In [39]:
weeks = pd.read_excel(masterdata, sheet_name="weeks")
weeks["Weight"] = weeks["Weight"].rolling(2, min_periods=1).mean()  # smoothing
weeks["Weight_normalized"] = weeks["Weight"] / weeks["Weight"].sum() * len(weeks)
weeks.set_index(keys="Week", inplace=True)
weeks.sample()

,Weight,Reason,Weight_normalized
Week,,,
17,1.0,NaN,0.385714


In [40]:
years = pd.read_excel(masterdata, sheet_name="years")
years.sample()

,Year,Growth
2,2022,1


### Customers
To find out the available customers and master data in S4, use this (remove comments when executing in Transaction DB02)
``` sql
SELECT but000.client, but000.partner, but000.name_org1, adrc.country, adrc.city1
FROM but000
JOIN but020 ON but000.client = but020.client AND but000.partner = but020.partner
JOIN adrc ON but020.client = adrc.client AND but020.addrnumber = adrc.addrnumber
JOIN but100 ON but000.client = but100.mandt AND but000.partner = but100.partner
WHERE but000.client = 202    -- partner id depends on the client
AND but000.bu_sort1 = '334'  -- any search term except 000, which also contains dirty student BP
AND but100.rltyp = 'FLCU00'  -- a customer role
ORDER BY but000.name_org1
```

Outside S4, we can only identify a customer by its name (the business partner in S4 depends on the system and client).

In [41]:
customers = pd.read_excel(masterdata, sheet_name="customers")
customers.sample()

,CNAME,COUNTRY,CITY,CWeight,VKORG,VTWEG,SPART
7,DC Bikes,US,Washington DC,10,UE00,WH,BI


### Materials

Find out available material master in S4
``` sql
SELECT mara.mandt, mara.matnr, mara.matkl, mara.mtart, makt.maktx
FROM mara
JOIN makt ON mara.mandt = makt.mandt AND mara.matnr = makt.matnr
WHERE mara.mandt = 202              -- client
AND mara.mtart IN ('FERT', 'HAWA')  -- finished and trading goods
AND mara.matnr LIKE '%888%'         -- any group
AND makt.spras = 'E'                -- English
ORDER BY mara.matnr
```

In [42]:
materials = pd.read_excel(masterdata, sheet_name="materials")
materials.sample()

,MATNR,MATKL,MTART,MAKTX,MWeight,Salesprice
1,CAGE1000,UTIL,HAWA,Water Bottle Cage,3.0,18.0


# Generator

### Working days

Produces subsequent dates
- only business days (no weekends)
- range is defined above

In [43]:
def gen_date():
    for d in pd.bdate_range(start=start_date, end=end_date, inclusive='left'):
        yield d

### Customers

Produce subsequent customers for a day
- number of customers = number of orders for that day is 50 at average
- number of orders varies by seasonality (C01); since seasonality also affects quantities, we only use sqrt() of the effect here
- The S4 system will create its own internal document numbers, but we also need an additional private (external to S4) number.
  This allows us to recognize our orders when implementing the daily delta logic later.
  The order number contains the date of the order plus a subsequential number to make it unique.
- Customers are selected according to their CWeight (C02); this also affects the number of order positions, hence, we only apply sqrt() of the weights.

In [44]:
def season_factor(day: pd.Timestamp):
    return math.sqrt(weeks.loc[day.week]["Weight_normalized"])

def gen_ordernum_with_customer(day: pd.Timestamp):
    # number of orders
    nof_orders = math.ceil(50 * season_factor(day))
    # customers
    customers_sample = customers.sample(n=nof_orders, replace=True, weights=customers["CWeight"].apply(math.sqrt))
    gen_customer = (c for (i, c) in customers_sample.iterrows())
    # produce result
    for x in range(nof_orders):
        order_number = f"{day.strftime('%Y-%m-%d')}#{x:03d}"
        yield(order_number, next(gen_customer))

### Materials

- The average nunber of positions is approx. 10
- The number of positions varies by sqrt() of customer weights (C02) - see above
- Materials are selected according to their MWeight (C04); again this also affects the quantity, why we apply sqrt()

In [45]:
def gen_material(day, customer):
    # positions
    factor = math.sqrt(customer["CWeight"] / customers["CWeight"].sum() * len(customers))
    nof_positions = math.ceil(10 * factor)
    # materials
    materials_sample = materials.sample(n=nof_positions, replace=True, weights=materials["MWeight"].apply(math.sqrt))
    gen_material = (m for (i, m) in materials_sample.iterrows())
    # produce result
    for pos in range(10, nof_positions * 10, 10):  # position numbers 10, 20, 30, ...
        yield(pos, next(gen_material))

### Quantity

- The average quantity is approx. 10
- The quantity depends on the sqrt() of seasonality (C01) - see above
- The quantity depends on the sqrt() material weights (C04) - see above
- The quantity tends to grow over time (C07)

In [46]:
def ret_quantity(day, material):
    factor = math.sqrt(material["MWeight"] / materials["MWeight"].sum() * len(materials))
    growth = years["Growth"].cumsum().loc[years["Year"] == day.year]
    quantity = math.ceil(10 * season_factor(day) * factor * growth)
    return (quantity, 'ST')

### Generate
Create list of sales orders by asking the respective oracles for customers, materials...  
(one year with 100_000+ orders takes 4 minutes on my laptop)

In [47]:
order_list = []
for day in gen_date():
    for (order_number, customer) in gen_ordernum_with_customer(day):
        for (position, material) in gen_material(day, customer):
            (quantity, uom) = ret_quantity(day, material)
            o = pd.concat([
                pd.Series(order_number, index=["Ext order id"]),
                pd.Series(position, index=["Position"]),
                pd.Series(day, index=["Day"]),
                customer.drop(labels=["CWeight"]),
                material.drop(labels=["MWeight"]),
                pd.Series([quantity, uom], index=["Quantity", "UoM"])
                ])
            order_list.append(o)

In [48]:
# convert to DataFrame
orders = pd.concat(order_list, axis='columns').T

In [49]:
orders.sample(3)

,Ext order id,Position,Day,CNAME,COUNTRY,CITY,VKORG,VTWEG,SPART,MATNR,MATKL,MTART,MAKTX,Salesprice,Quantity,UoM
3487,2022-12-22#032,50,2022-12-22,SoCal Bikes,US,Irvine,UW00,WH,BI,SHRT1000,UTIL,HAWA,T-shirt,30.0,14,ST
1507,2022-12-15#027,80,2022-12-15,Beantown Bikes,US,Boston,UE00,WH,BI,DXTR3000,BIKES,FERT,Deluxe Touring Bike (red),3000.0,16,ST
5182,2022-12-30#004,30,2022-12-30,Silicon Valley Bikes,US,Palo Alto,UW00,WH,BI,RHMT1000,SFTY,HAWA,Road Helmet,50.0,12,ST


In [50]:
orders.shape

(7494, 16)

### Save

In [51]:
orders.columns

Index(['Ext order id', 'Position', 'Day', 'CNAME', 'COUNTRY', 'CITY', 'VKORG',
       'VTWEG', 'SPART', 'MATNR', 'MATKL', 'MTART', 'MAKTX', 'Salesprice',
       'Quantity', 'UoM'],
      dtype='object')

The data file is made for import with ABAP into S/4:
- no header line
- tabulator as separator
- ABAP date format

In [69]:
columns = ['Ext order id', 'Position', 'Day', 'CNAME', 'VKORG', 'VTWEG', 'SPART', 'MATNR', 'Quantity', 'UoM']
with zipfile.ZipFile("../data/GlobalBikeABAP.zip", "w") as zf:
    for year in orders["Day"].dt.year.unique():
        file = f'{year:04d}GlobalBikeABAP.tsv'
        with zf.open(file, "w") as buffer:
            (
                orders.loc[orders["Day"].dt.year == year]
                .to_csv(buffer, columns=columns, index=False, header=False, sep='\t', date_format='%Y%m%d')
            )

------------
# Add financial information not relevant for S/4HANA

## Preparations
### Internal price
The internal price is calculated from the sales price using a fixed margin per product category.

In [55]:
data = {
    "MATKL" : ['BIKES', 'SFTY', 'UTIL'],
    "CostPortion" : [0.6, 0.5, 0.45]
}
matkl = pd.DataFrame(data)
matkl

,MATKL,CostPortion
0,BIKES,0.60
1,SFTY,0.50
2,UTIL,0.45


In [56]:
materials["Internalprice"] = (materials["Salesprice"] * materials.merge(matkl, left_on="MATKL", right_on="MATKL")["CostPortion"]).round(2)

In [57]:
materials.head()

,MATNR,MATKL,MTART,MAKTX,MWeight,Salesprice,Internalprice
0,BOTL1000,UTIL,HAWA,Water Bottle,1.0,20.0,9.0
1,CAGE1000,UTIL,HAWA,Water Bottle Cage,3.0,18.0,8.1
2,DGRB2000,UTIL,FERT,Deluxe GPS-Bike Computer Royal Blue,0.5,NaN,NaN
3,DGRR2000,UTIL,FERT,Deluxe GPS-Bike Computer Carmine Red,0.2,NaN,NaN
4,DGRW2000,UTIL,FERT,Deluxe GPS-Bike Computer Silver White,0.5,NaN,NaN


### Discounts

In [58]:
bins = [0, 10, 20, 100]  # np.inf would be the correct upper bound - 100 will do
labels = ['0', '10', '20']
discounts = {'0': 0, '10':0.03, '20':0.05}
customers["Discount%"] = pd.cut(customers["CWeight"], bins=bins, labels=labels, right=False).map(discounts).astype(float)

In [59]:
customers.head()

,CNAME,COUNTRY,CITY,CWeight,VKORG,VTWEG,SPART,Discount%
0,Airport Bikes,DE,Neu-Isenburg,10,DS00,WH,BI,0.03
1,Alster Cycling,DE,Hamburg,10,DN00,WH,BI,0.03
2,Bavaria Bikes,DE,München,20,DS00,WH,BI,0.05
3,Beantown Bikes,US,Boston,20,UE00,WH,BI,0.05
4,Big Apple Bikes,US,New York City,12,UE00,WH,BI,0.03


## Compute financial data

In [60]:
orders["Revenue"] = (orders["Salesprice"] * orders["Quantity"]).astype(float)

In [61]:
orders["Currency"] = orders["COUNTRY"].map({'DE':'EUR', 'US':'USD'})

In [62]:
# After merge(), the original index of orders is gone ;-(
# Hence, we need to preserve the index to be able to align with orders afterwards
orders["index"] = orders.index

In [63]:
# join with lookup tables
discount_pct = orders.merge(customers, on="CNAME").set_index(keys="index")["Discount%"].round(2)
orders["Discount"] = orders["Revenue"] * discount_pct

In [64]:
int_price = orders.merge(materials, on="MATNR").set_index(keys="index")["Internalprice"]
orders["Cost"] = orders["Quantity"] * int_price

Currency conversion?

In [65]:
orders.sample(1)

,Ext order id,Position,Day,CNAME,COUNTRY,CITY,VKORG,VTWEG,SPART,MATNR,...,MTART,MAKTX,Salesprice,Quantity,UoM,Revenue,Currency,index,Discount,Cost
3196,2022-12-21#038,10,2022-12-21,Furniture City Bikes,US,Grand Rapids,UE00,WH,BI,DXTR3000,...,FERT,Deluxe Touring Bike (red),3000.0,14,ST,42000.0,USD,3196,0.0,18900.0


## Save

In [66]:
orders.columns

Index(['Ext order id', 'Position', 'Day', 'CNAME', 'COUNTRY', 'CITY', 'VKORG',
       'VTWEG', 'SPART', 'MATNR', 'MATKL', 'MTART', 'MAKTX', 'Salesprice',
       'Quantity', 'UoM', 'Revenue', 'Currency', 'index', 'Discount', 'Cost'],
      dtype='object')

In [67]:
orders["SO Date"] = orders["Day"].dt.date # remove time portion

In [115]:
# no direct way in Python to have German number formatting
def format_comma(number: float):
    return f'{number}'.replace('.', ',')

orders["Revenue"] = orders["Revenue"].astype("float64")
orders["Discount"] = orders["Discount"].astype("float64")
orders["Cost"] = orders["Cost"].astype("float64")

In [118]:
columns = [
    'Ext order id', 'Position', 'SO Date',
    'CNAME', 'COUNTRY', 'CITY', 'VKORG', 'VTWEG', 'SPART',
    'MATNR', 'MATKL', 'MTART', 'MAKTX',
    'Quantity', 'Revenue', 'Currency', 'Discount', 'Cost']
with zipfile.ZipFile("../data/GlobalBikeAnalysis.zip", "w") as zf:
    for year in orders["Day"].dt.year.unique():
        file = f'{year:04d}GlobalBikeAnalysis.csv'
        with zf.open(file, "w") as buffer:
            (
                orders.loc[orders["Day"].dt.year == year]
                .to_csv(buffer, columns=columns, index=False, sep=';', float_format=format_comma) # German
                #.to_csv(buffer, columns=columns, index=False) # English
                #.to_excel(file, columns=columns, index=False)
            )